# 深度学习的使用

## 1. 配置机器学习应用

### 1.1 训练集、开发集、测试集

如前所述，应用机器学习/深度学习是一个迭代的过程，需要设定层数、隐藏层神经元数、学习速率、激活函数等一系列超参。

不同领域（NLP，CV）的超参设定，可能会有一些经验性的法则在里面，但通常离开相应的领域，即使应用同样的模型，超参设定的经验性法则也会不再试用。超参的最终结果，可能取决于一大批因素，比如训练集的大小、特征的数量、CPU/GPU的设定等等。可以说，在超参的设定上，并没有银弹，不存在一上来就能准确找到超参的办法，只能通过跑出的模型进行测试，再进一步迭代。

而将数据集分为训练集、开发集、测试集三部分，将有助于更高效地完成整个迭代过程。

传统机器学习的时代，最佳实践通常会将数据集分为三部分，60%的训练集，20%的开发集（也称交叉验证集），20%的测试集。整个工作流程是，不断地在训练集上训练模型，用开发集来测试哪个模型的表现最好，最终在实验了足够多次得到了一个足够好的模型后，使用测试集的评分来作为最终模型预测结果的无偏估计。**当不需要报告最终的无偏估计时，也可以仅仅分为70%的训练集和30%的测试集（实际上是开发集，不过按照惯例这里会叫做测试集，不是很好的命名规范）**。

而在如今的大数据时代，比如说我们可以获得百万条甚至更多的数据，由于开发集和测试集仅仅用于评分，只要能保证评分的稳定性，实际上并不需要20%这么多的数据，可能有10000条这样的量级就足够了。

大数据时代的另一个趋势是，人们常常在分布不一致的训练集和测试集上进行模型训练。比如猫的图片分类器，训练集的数据可能是从网上爬取下来的，分辨率较高，取景也相对专业；而测试集的数据来自用户通过APP进行上传获取，相对比较模糊，拍摄不太讲究。这样这两部分数据的分布就会不一致。这里的指导原则是**至少确保开发集和测试集的数据来自同一分布**。另一方面，由于深度学习对数据量的要求非常高，业界的一个趋势确实也是，想尽一切办法创造更多的训练数据，即便训练集会因此来自不同的分布。

### 1.2 偏差、方差

偏差和方差是机器学习中易学难精的一组概念，高级机器学习从业人员往往对其都有非常深刻的理解。

在二维的情况下，偏差和方差的情况，可以比较容易地可视化展现出来：

![Bias and Variance](img/Bias and Variance.png)

而在高维的情况下，偏差和方差则需要借助一些指标来进行判断：
1)训练误差<开发误差 & 训练误差接近人类专家可以达到的水平：高方差；
2）训练误差约等于开发误差 & 二者都远高于人类专家可以达到的水平：高偏差；
3）训练误差<开发误差 & 训练误差远高于人类专家可以到达的水平（也就意味着开发误差离人类专家的水平差距更大）：高偏差 & 高方差；
4）训练误差约等于开发误差 & 二者都接近于人类专家可以达到的水平：低偏差 & 低方差。

人类专家可以达到的误差水平，专业术语叫做**最优误差**或**贝叶斯误差**。在下面这个猫图片识别的例子里，我们认为最优误差约等于0。在这个前提下，会有图中的一些判定。（但是在某些实际问题中，比如用来识别的图片很模糊，可能很难确定最优误差大概是多少，那么具体的分析又会有不同；包括当训练集和开发集来自不同的分布，具体的分析也会有不一样；这些后面会介绍）

![Bias and Variance Cat classification.png](img/Bias and Variance Cat classification.png)

高方差、高偏差的情况意味着，分类器不仅没有学到训练集和开发集共同的模式，反而还受到了训练集噪声的影响，学到了不需要的模式。这种情况在高维数据的情况下也比较常见。
![High bias and high variance](img/High bias and high variance.png)

### 1.3 机器学习配方

在训练了一个初始模型后，你可以按照这个配方，来进行下面的诊断，并执行相应的修复步骤：

1）通过训练集误差，判断是否存在高偏差问题，如果不存在，也即是说模型对训练拟合情况已经非常好，可以直接进入下一步，如果存在：a) 尝试更大的神经网络（更多的隐藏层，更多的神经元），这个策略几乎总是管用，除非这个问题的贝叶斯误差就很大，也就是说人类专家也无法很好地判断；b) 尝试更多的迭代次数，这个策略至少不会让模型变得更糟；c) 尝试其它的神经网络结构，这个策略可能有用，可能没有用。

2）通过开发集误差，判断是否存在高方差问题，上面已经在训练集表现不错的模型，能否在开发集上也获得不错的泛化误差。如果存在：a) 尝试获得更多的数据，这个策略只会有正向作用；b) 使用正则化策略，控制过拟合。c) 和上面一样，尝试其它的神经网络结构，这个策略可能有用，可能没有用。

3）1和2的步骤可能会迭代多次，直到找到一个模型，在训练集和开发集的表现都很好，也即同时具备低偏差和低方差。

在深度学习时代，一个与以往的不同是，**人们不再过多地谈论偏差方差权衡**，在前深度学习时代，我们没有太多的工具可以单独减少偏差而不引起方差上升，或单独减少方差而不引起偏差上升。而深度学习时代/大数据时代，偏差（只要你可以训练更大的神经网络，并合理地进行正则化）和方差（只要你可以获得更多的数据）更多情况下变成了两个可以单独解决的问题，而不再经常遇到此消彼长的情况。

![Basic recipe for machine learning](img/Basic recipe for machine learning.png)

## 2. 正则化神经网络

### 2.1 正则化

解决高方差/过拟合，最直接的办法就是正则化。相比于获取更多数据可能需要花费过高的成本，正则化简单可行。

就逻辑回归的例子而言，正则化参数时，相比于高维的w，b通常可以省略掉。（是否省略对结果的影响不大）

L1正则化的结果相比L2会更稀疏，意味着更多w的值会变为0。L1正则化使模型参数稀疏之后，有助于压缩模型，节省内存。但在实际操作中，这方面的提升不会特别大。所以L1正则化的使用频率一般不高，至少不太会是出于压缩模型的考虑。绝大多数情况下，人们还是使用L2正则化。

引入正则化概念的同时，也会引入正则化系数 $\lambda$，这是又一个需要调的超参。

![Logistic regression regularization](img/Logistic regression regularization.png)

而对于神经网络来说，成本函数里的正则项，是所有权重矩阵中所有元素的平方和。用线性代数的术语描述，每个矩阵所有元素的平方和，称为矩阵的Frobenius范数，而正则项就可以描述为所有权重矩阵的Frobenius范数之和。

由于成本函数增加了正则项，计算梯度就也需要相应地增加正则项的梯度。由于下图里可以看到的变换，每次梯度迭代，权重矩阵都会首先变小（再按照没有正则化情况下的梯度进行更新），因此L2正则化，也称为**权重衰减 weight decay**。

![Neural Network regularization](img/Neural Network regularization.png)

### 2.2 为什么正则化可以减少过拟合

1）$\lambda$ 很大时，权重变小，极端情况下接近0，从消除了对应神经元的影响力，使得模型变得更简单。

2）$\lambda$ 很大时，权重变小，从而线性组合出来的 $Z$ 也会接近0。而以tanh激活函数为例，在输入接近为0的情况下，tanh函数的形状约等于线性函数。之前讨论过，如果每一层都是线性激活层，那么多复杂的神经网络都仅仅是线性函数。这样我们使用一个约等于线性函数的激活函数，模型也因此变得简单。

### 2.3 Dropout正则化

Dropout是针对神经网络的一种正则化技巧。在训练模型时，针对每一个训练样本，以一定概率随机将一些神经元的激活值置为0，进行正向和反向传播。不同的样本，随机取消的神经元不同。

常用的Dropout正则化实现，是inverted dropout：
![implementing inverted dropout](img/implementing inverted dropout.png)

注意到上面这个算法，训练时最终通过除以keep_prob这个参数进行了标准化。从而在开发/测试集上进行预测时，就不需要再另做改动，直接按照正常的流程正向传播即可。

### 2.4 了解Dropout

Dropout的正则化解释：

1）Dropout过程随机去掉了神经元，使整个神经网络变小，从而达到了正则化的目的。

2）每个神经元，作为下一层的输入，都是可能由于Dropout而被取消的。这样下一层的神经元要发挥激活作用，就会有正向的激励，不过分依赖于单个神经元，而是将权重尽可能分散。

另外一个实现细节是，keep_prob对于不同层来说，可以是不一样的。某些权重矩阵较大的层，由于更担心其过拟合，keep_prob的参数可以适当调小。输入层也可以设置dropout，但这个不常用。输出层通常keep_prob会设为1。对每一层都设置keep_prob值又有可能引入了过多的超参，所以有时也仅仅设置一个keep_prob值，然后指定哪些层使用dropout。

计算机视觉问题中，由于图片的像素点极多，也就是特征太多，相比而言样本通常都不够多，这时Dropout正则化几乎是一个默认的技巧。而对于其它应用来说，请牢记Dropout是一个正则化技巧，只在过拟合，又无法获得更多数据时使用。

在使用了Dropout之后，引发的一个问题是，之前成本函数 $J$ 不再是一个合理定义，用来衡量模型表现的指标。$J$ 未必会随着每次迭代都严格下降。

### 2.5 其它正则化方法

**样本生成 Data Augmentation**：比如对于计算机视觉问题，可以对图片进行水平翻转，从而获得新样本。也有随机旋转图片，然后再缩放生成新样本的方法。这些样本生成的方法，肯定不如全新的图片带来的效果好，但是可以节省收集新数据的成本。

![Data augmentation](img/Data augmentation.png)

**早停法 Early Stopping**：绘制训练误差和开发误差关于梯度下降迭代次数的曲线。在开发误差的最低点，提前结束迭代。提前结束迭代时得到的参数矩阵，通常Frobenius范数会处在一个适中的程度。早停法的一个弊端是，它将优化成本函数，和寻找超参控制过拟合两个过程耦合在了一起。一般情况下，我们会在某些超参空间内搜索，拿到一组确定的超参，优化（训练集）成本函数到其最小值，最后比较多组超参各自优化后的（开发集）成本函数最小值，挑选出全局最小的那个。而早停法中，每一组超参，都还没有达到训练集成本函数最小值，就停止迭代了。

![Early stopping](img/Early stopping.png)

## 3. 配置优化问题

### 3.1 标准化输入

标准化的流程，对每个特征，减去特征的均值，再除以平方差（PPT里和视频中说除以方差，和习惯不一致，意义不明）。需要注意的点：对测试集进行标准化处理时，要使用训练集产生的均值和标准差。

![Normalizing training sets](img/Normalizing training sets.png)

对输入进行标准化，可以加快神经网络的训练过程。因为标准化后，成本函数的轮廓图会更像圆形，这样学习速率可以设置得相对大一点。

![Why normalize inputs](img/Why normalize inputs.png)

当输入的各个特征本身已经在同一量级的时候，标准化的帮助不太明显。但对输入特征使用标准化转换，也基本不会有负面影响，同时通常都会使算法训练更快。

### 3.2 梯度消失和梯度爆炸

以计算 $\hat{y}$ 为例，假设权重矩阵为对角矩阵，截距向量为0向量。在神经网络非常深的时候，最终的预测结果也会是权重矩阵的对角值仅仅的指数幂。这个值略小于1，结果会无限趋于0。这个值略大于1，结果也会非常大。

类似的分析，也可以用于梯度。当梯度非常大和梯度趋于0的时候，训练神经网络都是十分困难的。而这也是过去制约训练大型神经网络的一个重要因素。

![vanishing and exploding gradients](img/vanishing and exploding gradients.png)

### 3.3 深度神经网络的权重初始化

我们无法完全消除梯度消失和梯度爆炸现象，但通过权重初始化的一些技巧，可以减轻这些现象。一个直觉上的观察是，如果某一层的神经元越多，那么我们希望相应的权重矩阵的元素值越小。下图给出了ReLU函数、Tanh函数的权重初始化公式，以及一个普遍适用的公式。

![Single neuron example](img/Single neuron example.png)

### 3.4 梯度的数值逼近

通过较小的值（非无穷小量），来对梯度进行数值逼近，作为对反向传播中梯度下降实现的一个辅助检查工具。

![Checking your derivative computation](img/Checking your derivative computation.png)

### 3.5 梯度检查

![Gradient check for a neural network](img/Gradient check for a neural network.png)

![Gradient checking](img/Gradient checking.png)